In [1]:
# mount the Google Drive
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/DSA4212/dataset/assignment_2

Mounted at /content/drive
/content/drive/MyDrive/DSA4212/dataset/assignment_2


In [3]:
import jax as jx
from jax.config import config
config.update('jax_enable_x64', True)  # often needed for LBFGS that requires high-precision


import jax.numpy as jnp
import numpy as onp
import pylab as plt
import tqdm

import imageio
import os
from skimage.transform import rescale, resize, downscale_local_mean
import pandas as pd
import time
import scipy

try:
    import surprise
except ModuleNotFoundError: # Install supriise  if missing
    !pip install --quiet surprise
    import surprise
     

from surprise import NormalPredictor,KNNBasic
from surprise import Dataset
from surprise import Reader

from surprise.model_selection.validation import cross_validate

  Preparing metadata (setup.py) ... done


In [19]:
# Reading in data
# anime_id,name,genre,type,rating,members
anime_name = pd.read_csv("assignment_2_anime.csv")

# User_id,anime_id,rating
train = pd.read_csv("assignment_2_ratings_train.csv")
# User_id,anime_id,rating
test = pd.read_csv("assignment_2_ratings_test.csv")

In [9]:
#mse function 
def mse(pred,y):
  return onp.mean((pred-y)**2)

# User mean model

In [10]:
def user_mean_model(df_train,df_test):
  """
  Takes input as training data frame and test data frame
  finds the user mean from training data frame and apply as prediction for test data frame and find the relevant test mse 
  """
  user_mean= df_train.groupby('user_id')['rating'].mean().to_frame().reset_index()
  user_mean.rename(columns = {'rating':'prediction'},
          inplace = True)
  #merge prediction with test data
  df_test_with_pred=pd.merge(df_test,user_mean,how="left",on="user_id").fillna(0)
  #return the test mse 
  return mse(df_test_with_pred["prediction"],df_test_with_pred["rating"])

In [20]:
user_mean_model(train,test)

1.9264275564706208

# Anime mean model

In [12]:
def anime_mean_model(df_train,df_test):
  """
  Takes input as training data frame and test data frame
  finds the anime mean from training data frame and apply as prediction for test data frame and find the relevant test mse 
  """
  anime_mean= df_train.groupby('anime_id')['rating'].mean().to_frame().reset_index()
  anime_mean.rename(columns = {'rating':'prediction'},
          inplace = True)
  #merge prediction with test data
  df_test_with_pred=pd.merge(df_test,anime_mean,how="left",on="anime_id").fillna(0)
  #return the test mse 
  return mse(df_test_with_pred["prediction"],df_test_with_pred["rating"])

In [21]:
anime_mean_model(train,test)

2.066542133906232

# Global mean model

In [14]:
def global_mean_model(df_train,df_test):
  """
  Takes input as training data frame and test data frame
  finds the global mean from training data frame and apply as prediction for test data frame and find the relevant test mse 
  """
  mean= df_train['rating'].mean()

  

  return mse(mean,df_test["rating"])

In [22]:
global_mean_model(train,test)

2.472703843572946

# Normal approximation model

In [23]:

# Train data
reader = Reader(rating_scale = (1, 10))

train_data = Dataset.load_from_df(train[['user_id', 'anime_id', 'rating']], reader)

trainset = train_data.build_full_trainset()

# Test data
reader = Reader(rating_scale = (1, 10))

test_data = Dataset.load_from_df(test[['user_id', 'anime_id', 'rating']], reader)

testset = test_data.build_full_trainset().build_testset()
     


     





In [24]:
algo = NormalPredictor()
algo.fit(trainset)

In [26]:
predictions = algo.test(testset)
surprise.accuracy.mse(predictions)

MSE: 4.6138


4.6138226969959035